In this project, I used UV rather than PIP. See the readme.md for more information including how to download UV and then sync it into an application. UV is a high-performance alternative to pip that is designed to be a drop-in replacement for many workflows, offering significantly faster performance for package installation and dependency resolution. It also combines the functionality of pip, venv, and pip-tools into a single tool, with additional features like environment management and modern dependency resolution. Comparisons are as follows: 

| Feature | pip | uv |
|---------|-----|-----|
| **Speed** | Standard (Python-based) | 10-100x faster (Rust-based) |
| **Installation** | Bundled with Python | Separate install required |
| **Package Installation** | ✅ Yes | ✅ Yes |
| **Dependency Resolution** | Basic | Advanced, more reliable |
| **Virtual Environments** | Via separate `venv` module | Built-in creation & management |
| **Python Version Management** | ❌ No | ✅ Yes (installs Python versions) |
| **Lock Files** | ❌ No (community tools needed) | ✅ Yes (uv.lock) |
| **Parallel Downloads** | Limited | ✅ Yes |
| **Caching** | Basic | Advanced, cross-project |
| **Project Management** | ❌ No | ✅ Yes (similar to Poetry) |
| **requirements.txt Support** | ✅ Native | ✅ Compatible |
| **Maturity** | Established (2000s) | New (2024) |
| **Compatibility** | Universal | High, but newer |
| **Written In** | Python | Rust |
| **Use Case** | General purpose, legacy | Modern projects, speed-critical |
| **Learning Curve** | Minimal | Slightly steeper |
| **Community Adoption** | Universal standard | Growing rapidly |

There was an issue with UV sync because my VS Code environment didn't inherit the windows path. So I went to File\Preferences\Settings and searched for terminal.integrated.env.windows. This gave me the option to open this JSON file. You can set terminal.integrated.inheritEnv to true or add the path to terminal.integrated.env.windows as shown below.

```jsonc
{
    "python.defaultInterpreterPath": "c:\\Users\\damie\\AppData\\Local\\Programs\\Python\\Python313\\python.exe",
    "workbench.editor.empty.hint": "hidden",
    "explorer.compactFolders": false,
    "git.autofetch": true,
    "terminal.integrated.env.windows": {
        "PATH": "${env:PATH}"
    },
    "terminal.integrated.inheritEnv": false
}
```

# Basic Agentic Workflow

Helloooo everyone and welcome to an exciting lesson on Agentic AI! 🎉

Today, we're diving into **prompt chaining** agentic workflow pattern. What's that? It's just passing the output from one LLM to the next, step by step. Think of it like a relay race, but with prompts instead of batons.

We'll keep things super simple: manually run each cell, watch the magic happen, and see how chaining LLM calls lets us build more complex workflows.

Ready to see how agents can work together? Let's get started!

## As always, libraries first!

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown


load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key")

You can set your API Keys for each of the LLM providers using the following links:

- [OpenAI](https://platform.openai.com/api-keys)
- [Anthropic](https://console.anthropic.com/settings/keys)
- [Gemini](https://aistudio.google.com/app/apikey)

Once you have created the API Keys, you can store them on your `.env` file at the root of this repo

<div style="border-radius:16px;background:#2e3440;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
    <b style="color:#88c0d0;font-size:1.25em">Info:</b>
    <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
        <li>You can complete this entire notebook using just OpenAI models if you prefer!</li>
        <li>It's absolutely fine to skip Anthropic and Gemini for now — the workflow works perfectly with only OpenAI.</li>
        <li>Feel free to experiment with other providers later, but don't let missing API keys slow you down.</li>
    </ul>
    <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#88c0d0;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💡</div>
</div>

## The Workflow

```mermaid
graph LR
    A[Generate Tickets] --> B[Classify Priority] --> C[Respond to Tickets]
```

## Lets start with using OpenAI

In [3]:
# client
openai_client = OpenAI()

In [4]:
# messages list
message = "I want you to generate a customer support ticket for a 3rd party re-seller. "
message += "The ticket should be a single sentence describing a common issue a customer might face with their product or service. "
message += "Please ensure the ticket is varied and covers different types of problems. "
message += "Do not include a subject, only the body of the ticket."

messages = [{"role": "user", "content": message}]

In [5]:
# response

openai_response = openai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

ticket = openai_response.choices[0].message.content
# print(f"### Generated Ticket:\n{ticket}")
display(Markdown(f"### Generated Ticket:\n{ticket}"))

### Generated Ticket:
The customer is unable to activate the software license due to an error message during installation.

I love markdown. It is a lightweight method of rendering and formatting text that is super versatile without having to use heavy softwares like MS Word or Google Docs.

You can learn more about markdown sytanx [here](https://www.markdownguide.org/basic-syntax/)

A really informative YouTube video talking about the [Unreasonable Effectiveness of Plain Text](https://www.youtube.com/watch?v=WgV6M1LyfNY)

## Lets pass these on to an Anthropic model and ask it to classify the priority level of each ticket

In [ ]:
# anthropic client. Note we are using the OpenAI wrapper for Anthropic by passing in the base_url
anthropic_client = OpenAI(api_key=ANTHROPIC_API_KEY, base_url="https://api.anthropic.com/v1")

In [7]:
# messages list
message = "I want you to classify the priority of the following customer support ticket. "
message += "The ticket is as follows: "+ ticket + " "
message += "Please classify the priority as either 'Low', 'Medium', or 'High'. "
message += "Respond with only the priority level."

messages = [{"role": "user", "content": message}]

In [8]:
# response

anthropic_response = anthropic_client.chat.completions.create(
    model="claude-3-5-haiku-latest",
    messages=messages
)

priority = anthropic_response.choices[0].message.content
display(Markdown(f"### Classified Priority:\n{priority}"))

### Classified Priority:
High

## Now Gemini should determine the appropriate response

In [ ]:
# gemini client. Note we are using the OpenAI wrapper for Gemini by passing in the base_url
gemini_client = OpenAI(api_key=GEMINI_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [10]:
# messages list

message = "You are to determine an appropriate response to the following customer support ticket. "
message += "The ticket is as follows: "+ ticket + " "
message += "The priority level of this ticket is: " + priority + " "
message += "Please provide a response that addresses the customer's issue in a short and concise manner. "
    
messages = [{"role": "user", "content": message}]

In [11]:
# response

gemini_response = gemini_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=messages
)

response = gemini_response.choices[0].message.content
display(Markdown(f"### Generated Response:\n{response}"))

### Generated Response:
Subject: High Priority: Urgent Assistance Needed for License Activation Error

Dear [Customer Name],

We understand you're experiencing a high-priority issue with your software license activation during installation, and we apologize for the inconvenience this is causing.

To help us resolve this swiftly, please provide the exact error message you are receiving. Our technical team is ready to provide immediate support and will contact you directly with specific next steps, which may include a call or remote session.

Sincerely,
[Your Name/Support Team]

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
    <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
    <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6"></ul>
        <li>Recreate the customer support ticket workflow using an <b>evaluator-optimizer agentic workflow pattern</b> instead of prompt chaining.</li>
        <li>Your evaluator agent should assess the quality and completeness of each ticket and suggest improvements.</li>
        <li>Your optimizer agent should revise the tickets based on evaluator feedback, aiming for clarity and actionable details.</li>
        <li>Try to implement this using at least two LLM calls (one for evaluation, one for optimization) and display the before/after results.</li>
        <li>Share your work in the community-contributions folder by creating a folder with your name. Eg. shaheer-airaj.</li>
    </ul>
    <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>